# Regularization

Let's try to regularize the model applying some Lasso regressions, in order to shrink the coefficients of our model towards zero.

In [12]:
import numpy as np
from simple_linear_reg import *
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import RepeatedKFold, GridSearchCV

In [2]:
diamonds = pd.read_csv('../data/train_cate.csv')
diamonds.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.50,5,7,4,62.3,55.0,5.11,5.07,3.17,1845
1,1,1.54,2,2,5,63.6,60.0,7.30,7.33,4.65,10164
2,2,1.32,3,1,2,61.7,60.0,6.95,7.01,4.31,5513
3,3,1.20,5,2,3,62.1,55.0,6.83,6.79,4.23,5174
4,4,1.73,4,2,3,61.2,60.0,7.67,7.65,4.69,10957


### Simple Lasso regression

Let's apply first a Lasso regression to our raw data and get an idea on how the data could be improved.

In [9]:
X = diamonds.iloc[:,1:10]
y = diamonds.iloc[:,10]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [8]:
model = Lasso().fit(X_train, y_train)

In [9]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [10]:
lr_results(y_train, y_test, y_train_pred, y_test_pred)

,train_set,test_set
r2,0.909285,0.901092
mae,800.115018,819.195925
rmse,1202.564090,1281.475772


As we can see, Lasso regression to our raw data does not improve the results of our model that much. This could be because of the alpha level of the model, which by default is 1. Let's try now a new approach within a Lasso model with cross validation and a list of 100 different alpha values, from 0.01 to 1.

In [9]:
model = Lasso(normalize=True)

x_val = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)

grid = {'alpha': np.arange(0,1,0.01)}

model = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=x_val, n_jobs=-1).fit(X_train, y_train)

/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/model_selection/_search.py:765: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23967999220.713634, tolerance: 52122111.364090614
  model = cd_fast.enet_coordinate_descent(


In [10]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [11]:
lr_results(y_train, y_test, y_train_pred, y_test_pred)

,train_set,test_set
r2,0.908031,0.840495
mae,812.187481,817.450026
rmse,1217.025821,1595.747679


In [12]:
model.best_params_

{'alpha': 0.0}

Not a huge one, but we can see a small improvement on the RMSE that tells us maybe Lasso with a range of alphas and cross validation could show us a way of improvement.

Let's try now this very same models dropping those columns that were highly correlated when we tried the simple linear regression models. First, let's drop the columns related to the size of the diamonds.

In [13]:
X = diamonds.iloc[:,1:-4]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [15]:
model = Lasso()

x_val = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)

grid = {'alpha': np.arange(0,1,0.01)}

model = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=x_val, n_jobs=-1).fit(X_train, y_train)

/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/model_selection/_search.py:765: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24526264900.668846, tolerance: 51715801.57757731
  model = cd_fast.enet_coordinate_descent(


In [16]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [17]:
lr_results(y_train, y_test, y_train_pred, y_test_pred)

,train_set,test_set
r2,0.905150,0.903746
mae,857.267988,862.153509
rmse,1231.117795,1258.950541


In [18]:
model.best_params_

{'alpha': 0.0}

RMSE keeps dropping for the test set and increasing for the train, which means the model is getting less overfitted. $R^2$ has also increased for the test set. 

Let's try now dropping those features that were not statistically significant when we performed a OLS test in our simple linear regression notebook.

In [10]:
X = diamonds.iloc[:,1:-3]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [13]:
model = Lasso(normalize=True)

x_val = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)

grid = {'alpha': np.arange(0,1,0.01)}

model = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=x_val, n_jobs=-1).fit(X_train, y_train)

/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/model_selection/_search.py:765: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23935897632.854744, tolerance: 51986446.79746568
  model = cd_fast.enet_coordinate_descent(


In [14]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [15]:
lr_results(y_train, y_test, y_train_pred, y_test_pred)

,train_set,test_set
r2,0.907915,0.906401
mae,808.728880,796.641350
rmse,1216.210535,1228.736544


In [16]:
model.best_params_

{'alpha': 0.0}

### First attemp to Kaggle

After all this trys, is time to send a first attemp to the ranking, so it maybe could open my eyes to something better.

In [17]:
diamonds_pred = pd.read_csv('../data/pred_cate.csv')
diamonds_pred.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.45,5,6,3,62.8,58.0,4.88,4.84,3.05
1,1,1.23,5,3,3,61.0,56.0,6.96,6.92,4.23
2,2,0.33,5,2,8,61.8,55.0,4.46,4.47,2.76
3,3,0.51,5,7,4,58.0,60.0,5.29,5.26,3.06
4,4,0.40,5,6,4,62.2,59.0,4.71,4.74,2.94


In [18]:
X_pred = diamonds_pred.iloc[:,1:-2]

In [19]:
diamonds_pred['price'] = model.predict(X_pred)

In [20]:
submission_1 = diamonds_pred.iloc[:,[0,-1]]
submission_1.set_index('id', inplace=True)
submission_1.head()

,price
id,
0,924.469484
1,6780.010436
2,1403.221097
3,2385.851841
4,1061.152040


In [29]:
submission_1.to_csv('data/submission_1.csv')